In [ ]:
import random
import math
import numpy as np
from itertools import combinations
import plotly.graph_objects as go
import networkx as nx
from typing import List, Tuple

In [ ]:
def get_v_star(T):
    # Create an initial vector 'u' of zeros with length 'T'
    u = np.zeros(T)
    # Set the first element of vector 'u' to -1 and the last element to 1
    u[0] = -1
    u[-1] = 1
    # Initialize the list 'v_star' with the initial vector 'u'
    v_star = [u.copy()]
    
    # Loop over the length of 'u' minus one times to generate shifted versions of 'u'
    for i in range(T - 1):
        # Rotate 'u' by moving the last element to the front
        u = np.roll(u, 1)
        # Append the updated vector 'u' to the list 'v_star'
        v_star.append(u.copy())
    
    # Return 'v_star' as a list of lists, which is easier to process in the main function
    return np.array(v_star)

def create_neighbors_list(s: list[int]) -> (list[int], list[int]):
    """
    Create a set of pairs of schedules that are from the same neighborhood.
    
    Parameters:
      s (list[int]): A list of integers with |s| = T and sum N.
      
    Returns:
      tuple(list[int], list[int]): A pair of schedules.
    """
    # Create a set of vectors of length T
    v_star = get_v_star(len(s))
    
    # Choose a random element of t with probability P(t = i) = C(T,i)/((2^T)-2) for i in [1, ..., T-1]
    i = random.choices(range(1, len(s)), weights=[math.comb(len(s), i) for i in range(1, len(s))])[0]
    
    # Create a list l of all subsets of t with length i
    l = list(combinations(range(len(s)), i))
    
    # Choose a random element of l with probability 1/|l| and save it as j
    j = random.choice(l)
    
    # Select all elements of V* with index in j and save them as V_j
    # Sum the corresponding vectors in v_star
    v_j = [v_star[idx] for idx in j]  # Convert NumPy arrays to lists
    
    # Sum the elements of v_j and s and save as s_p
    s_p = s.copy()
    for v in v_j:
      s_p_temp = [int(x + y) for x, y in zip(s_p, v)]
      if np.all(np.array(s_p_temp) >= 0):
        s_p = s_p_temp
        
    return s, s_p

def test_create_neighbors_list():
    # Test case 1: Basic structure test (Length of s = 3)
    s = [1, 2, 1]
    s_original, s_p = create_neighbors_list(s)
    print(s_original, s_p)
    
    assert len(s_original) == len(s_p), "The two schedules must be of equal length"
    
    # Test case 2: Edge case with minimum length (Length of s = 2)
    s = [1, 1]
    s_original, s_p = create_neighbors_list(s)
    
    assert len(s_original) == len(s_p), "The two schedules must be of equal length for small inputs"
    assert sum(s_original) == sum(s_p), "The sum of elements should remain the same before and after the transformation"
    
    # Test case 3: Test for larger list and verify no mutation of original list
    s = [3, 2, 4, 1]
    s_original, s_p = create_neighbors_list(s)
    
    assert len(s_original) == len(s_p), "The schedules should have the same length"
    assert sum(s_original) == sum(s_p), "The sum should not change"
    assert s == s_original, "The original list should not be modified"
    
    # Test case 4: Test with all elements zero (edge case)
    s = [0, 0, 0, 0]
    s_original, s_p = create_neighbors_list(s)
    print(s_original, s_p)
    
    assert len(s_original) == len(s_p), "The schedules should have the same length"
    assert s_p == s_original, "If all elements are zero, the result should be identical"
    
    # Test case 5: Probabilistic test
    s = [1, 2, 3, 4]
    counts = [0] * len(s)  # Corrected length of counts
    for _ in range(1000):
        s_original, s_p = create_neighbors_list(s)
        diff = [x - y for x, y in zip(s_p, s_original)]
        
        # Check if -1 is in diff to avoid ValueError
        if -1 in diff:
            i = diff.index(-1)
            counts[i] += 1
        else:
            # If no -1 in diff, the schedules are identical; you might want to handle this case
            pass

    print("Probabilistic distribution for i (should reflect C(T,i)/((2^T)-2)): ", counts)

test_create_neighbors_list()


In [ ]:
import numpy as np
from functions import service_time_with_no_shows

def compute_convolutions(probabilities, N, q=0.0):
    """
    Computes the k-fold convolution of a probability mass function for k in range 1 to N.

    Parameters:
    probabilities (list of floats): The PMF represented as a list where the index is the service time and the value is the probability.
    N (int): The maximum number of convolutions to compute.

    Returns:
    dict: A dictionary where keys are k and values are the convoluted service times (PMFs).
    """
    convolutions = {}
    result = probabilities.copy()
    result = service_time_with_no_shows(result, q)
    for k in range(1, N + 1):
        if k == 1:
            convolutions[k] = result
        else:
            result = np.convolve(result, probabilities)
            convolutions[k] = result
    return convolutions

def test_compute_convolutions():
    # Define a sample probability mass function
    # probabilities = [0.1, 0.2, 0.3, 0.4]  # Service times 0, 1, 2, 3
    probabilities = [0., 0.00599585, 0.00644478, 0.02427306, 0.04753406, 0.07355202, 0.16864656, 0.12338866, 0.10182152, 0.09778246, 0.07283463, 0.05794873, 0.05378825, 0.04945164, 0.04805953, 0.02797309, 0.02179657, 0.0183278, 0.0003808 ]
    N = 20
    convolutions = compute_convolutions(probabilities, N, q=0.0)

    # Test that the probabilities sum to 1 for each convolution
    for k in range(1, N + 1):
        conv_pmf = convolutions[k]
        total_prob = sum(conv_pmf)
        assert abs(total_prob - 1.0) < 1e-6, f"Total probability at k={k} does not sum to 1."
        print(f"Convolution at k={k}: {conv_pmf}")

    print("All tests passed.")

# Run the test function
test_compute_convolutions()

In [ ]:
from typing import List, Generator, Tuple

def calculate_objective_serv_time_lookup(schedule: List[int], d: int, q: float, convolutions: dict) -> Tuple[float, float]:
    """
    Calculate the objective value based on the given schedule and parameters using precomputed convolutions.

    This function uses precomputed convolutions of the service time distribution,
    starting from the 1-fold convolution (key 1) which contains the adjusted service time distribution.

    Parameters:
    schedule (List[int]): A list representing the number of patients scheduled in each time slot.
    d (int): Duration threshold or maximum allowed service time per slot.
    q (float): No-show probability.
    convolutions (dict): Precomputed convolutions of the service time distribution, with key 1 containing the adjusted service time distribution.

    Returns:
    Tuple[float, float]: 
        - ewt (float): The sum of expected waiting times.
        - esp (float): The expected spillover time (overtime).
    """
    sp = np.array([1], dtype=np.float64)  # Initial service process (no waiting time)
    ewt = 0  # Total expected waiting time

    for x in schedule:
        if x == 0:
            # No patients in this time slot
            # Adjust sp for the duration d (service process moves ahead)
            sp_new = []
            sp_new.append(np.sum(sp[:d + 1]))
            sp_new.extend(sp[d + 1:])
            sp = np.array(sp_new)
        else:
            # Patients are scheduled in this time slot
            wt_temp = [sp.copy()]
            # Add expected waiting time for the first patient
            ewt += np.dot(range(len(sp)), sp)
            # For each additional patient
            for i in range(1, x):
                # The waiting time distribution for the ith patient is the convolution
                # of the previous patient's waiting time with s (adjusted service time distribution)
                conv_s = convolutions.get(1)  # Adjusted service time distribution
                wt = np.convolve(wt_temp[i - 1], conv_s)
                wt_temp.append(wt)
                ewt += np.dot(range(len(wt)), wt)
            # Update sp for the next time slot
            conv_s = convolutions.get(1)  # Adjusted service time distribution
            sp = np.convolve(wt_temp[-1], conv_s)
            # Adjust sp for duration d
            sp_new = []
            sp_new.append(np.sum(sp[:d + 1]))
            sp_new.extend(sp[d + 1:])
            sp = np.array(sp_new)
    # Expected spillover time
    esp = np.dot(range(len(sp)), sp)
    return ewt, esp
  
def calculate_objective(schedule: List[int], s: List[float], d: int, q: float) -> Tuple[float, float]:
    """
    Calculate the objective value based on the given schedule and parameters.

    This function adjusts the service times distribution for no-shows, calculates 
    the waiting times for all patients in the schedule, sums the expected 
    waiting times, and calculates the spillover time for the last interval (overtime).

    Parameters:
    schedule (List[int]): A list representing the number of patients scheduled in each time slot.
    s (List[float]): Service times probability distribution.
    d (int): Duration threshold or maximum allowed service time per slot.
    q (float): No-show probability.

    Returns:
    Tuple[float, float]: 
        - ewt (float): The sum of expected waiting times.
        - esp (float): The expected spillover time (overtime).
    """
    # Adjust the service time distribution for no-shows
    s = service_time_with_no_shows(s, q)
    # Initialize the service process (probability distribution of waiting times)
    sp = np.array([1], dtype=np.float64)
    wt_list = []
    ewt = 0  # Expected waiting time
    for x in schedule:
        if x == 0:
            # No patients in this time slot
            wt_temp = [np.array(sp)]
            wt_list.append([])
            sp = []
            sp.append(np.sum(wt_temp[-1][:d + 1]))
            sp.extend(wt_temp[-1][d + 1:])
        else:
            # Patients are scheduled in this time slot
            wt_temp = [np.array(sp)]
            # Add expected waiting time for the first patient
            ewt += np.dot(range(len(sp)), sp)
            # For each additional patient
            for i in range(x - 1):
                # Convolve the waiting time with the service time distribution
                wt = np.convolve(wt_temp[i], s)
                wt_temp.append(wt)
                # Add expected waiting time
                ewt += np.dot(range(len(wt)), wt)
            wt_list.append(wt_temp)
            # Update the service process for the next time slot
            sp = []
            convolved = np.convolve(wt_temp[-1], s)
            sp.append(np.sum(convolved[:d + 1]))
            sp.extend(convolved[d + 1:])
    # Calculate expected spillover time
    esp = np.dot(range(len(sp)), sp)
    return ewt, esp
  
def test_calculate_objective_serv_time_lookup_with_timing():
    """
    Test the calculate_objective_serv_time_lookup function by comparing its outputs
    and execution time with the original calculate_objective function.

    The test verifies that both functions produce the same expected waiting time (ewt)
    and expected spillover time (esp) for a sample schedule and parameters, and compares
    their execution times.
    """
    import numpy as np
    import time

    # Sample service time distribution (probabilities)
    s = [0., 0.00599585, 0.00644478, 0.02427306, 0.04753406, 0.07355202, 0.16864656, 0.12338866, 0.10182152, 0.09778246, 0.07283463, 0.05794873, 0.05378825, 0.04945164, 0.04805953, 0.02797309, 0.02179657, 0.0183278, 0.0003808 ]

    # Normalize the service time distribution to sum to 1
    total_prob = sum(s)
    s = [prob / total_prob for prob in s]

    # Sample schedule: Number of patients scheduled in each time slot
    schedule = [4, 2, 3, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 2, 0, 3, 0, 4]

    # Parameters
    d = 5       # Duration threshold or maximum allowed service time per slot
    q = 0.1      # No-show probability

    # Precompute convolutions for the maximum number of patients in any time slot
    N = max(schedule)
    convolutions = compute_convolutions(s, N, q)

    # Measure execution time for the original function
    start_time_original = time.time()
    ewt_original, esp_original = calculate_objective(schedule, s, d, q)
    end_time_original = time.time()
    time_original = end_time_original - start_time_original

    # Measure execution time for the lookup function
    start_time_lookup = time.time()
    ewt_lookup, esp_lookup = calculate_objective_serv_time_lookup(schedule, d, q, convolutions)
    end_time_lookup = time.time()
    time_lookup = end_time_lookup - start_time_lookup

    # Define a tolerance for floating-point comparisons
    tolerance = 1e-6

    # Check if the expected waiting times are approximately equal
    ewt_match = abs(ewt_original - ewt_lookup) < tolerance
    # Check if the expected spillover times are approximately equal
    esp_match = abs(esp_original - esp_lookup) < tolerance

    # Print the results
    print("Original Function Output:")
    print(f"Expected Waiting Time (ewt): {ewt_original}")
    print(f"Expected Spillover Time (esp): {esp_original}")
    print(f"Execution Time: {time_original:.6f} seconds\n")

    print("Lookup Function Output:")
    print(f"Expected Waiting Time (ewt): {ewt_lookup}")
    print(f"Expected Spillover Time (esp): {esp_lookup}")
    print(f"Execution Time: {time_lookup:.6f} seconds\n")

    # Assert that both outputs match within the defined tolerance
    assert ewt_match, "Expected Waiting Times do not match."
    assert esp_match, "Expected Spillover Times do not match."

    print("Test passed: The outputs of both functions match within the defined tolerance.")

    # Compare execution times
    if time_lookup > 0:
        speedup = time_original / time_lookup
        print(f"The lookup function is {speedup:.2f} times faster than the original function.")
    else:
        print("Execution time for the lookup function is too small to measure speedup.")

# Run the test function
test_calculate_objective_serv_time_lookup_with_timing()

In [ ]:
def build_welch_bailey_schedule(N, T):
    """
    Build a schedule based on the Welch and Bailey (1952) heuristic.

    Parameters:
    N (int): Number of patients to be scheduled.
    T (int): Number of time intervals in the schedule.

    Returns:
    list: A schedule of length T where each item represents the number of patients scheduled
          at the corresponding time interval.
    """
    # Initialize the schedule with zeros
    schedule = [0] * T

    # Schedule the first two patients at the beginning
    schedule[0] = 2
    remaining_patients = N - 2

    # Distribute patients in the middle time slots with gaps
    for t in range(1, T - 1):
        if remaining_patients <= 0:
            break
        if t % 2 == 1:  # Create gaps (only schedule patients at odd time slots)
            schedule[t] = 1
            remaining_patients -= 1

    # Push any remaining patients to the last time slot
    schedule[-1] += remaining_patients

    return schedule

def test_build_welch_bailey_schedule():
    """
    Test the build_welch_bailey_schedule function with multiple test cases.
    """
    # Test case 1: Simple case with N < T
    N, T = 10, 15
    schedule = build_welch_bailey_schedule(N, T)
    assert len(schedule) == T, f"Failed: Expected length {T}, got {len(schedule)}"
    assert sum(schedule) == N, f"Failed: Expected sum {N}, got {sum(schedule)}"
    assert schedule[0] == 2, "Failed: First slot should have 2 patients"
    assert all(schedule[i] == 0 for i in range(2, T - 1, 2)), "Failed: Gaps not created correctly"

    # Test case 2: Exact fit of N and T
    N, T = 7, 7
    schedule = build_welch_bailey_schedule(N, T)
    assert len(schedule) == T, f"Failed: Expected length {T}, got {len(schedule)}"
    assert sum(schedule) == N, f"Failed: Expected sum {N}, got {sum(schedule)}"

    # Test case 3: N > T
    N, T = 20, 10
    schedule = build_welch_bailey_schedule(N, T)
    assert len(schedule) == T, f"Failed: Expected length {T}, got {len(schedule)}"
    assert sum(schedule) == N, f"Failed: Expected sum {N}, got {sum(schedule)}"
    assert schedule[-1] > 1, "Failed: Excess patients not pushed to the last interval"

    # Test case 4: N is small
    N, T = 3, 10
    schedule = build_welch_bailey_schedule(N, T)
    assert len(schedule) == T, f"Failed: Expected length {T}, got {len(schedule)}"
    assert sum(schedule) == N, f"Failed: Expected sum {N}, got {sum(schedule)}"
    assert schedule[0] == 2, "Failed: First slot should have 2 patients"
    assert schedule.count(1) == 1, "Failed: Only one remaining patient should be scheduled"

    # Test case 5: N equals T
    N, T = 15, 15
    schedule = build_welch_bailey_schedule(N, T)
    assert len(schedule) == T, f"Failed: Expected length {T}, got {len(schedule)}"
    assert sum(schedule) == N, f"Failed: Expected sum {N}, got {sum(schedule)}"

    print("All test cases passed!")


# Run the test function
test_build_welch_bailey_schedule()

In [ ]:
def create_schedule_network_from_lists(
    schedules: List[List[int]],
    objective_values: List[float],
    echo: bool = False
) -> go.Figure:
    """
    Creates and visualizes a network graph of schedules with annotated edges.

    Parameters:
    - schedules (List[List[int]]): A list of schedules, each schedule is a list of patient allocations across time intervals.
    - objective_values (List[float]): A list of objective values corresponding to each schedule.
    - echo (bool): If True, prints schedule details and objective values.

    Returns:
    - fig (plotly.graph_objects.Figure): Interactive network graph.
    """
    
    # 1. Validate Inputs
    if len(schedules) != len(objective_values):
        raise ValueError("The number of schedules must match the number of objective values.")
    
    # 2. Convert schedules to tuples for immutability and as node identifiers
    schedule_tuples = [tuple(schedule) for schedule in schedules]
    
    # 3. Create a mapping from schedule to objective value
    schedule_to_obj = {schedule: obj for schedule, obj in zip(schedule_tuples, objective_values)}
    
    # 4. Initialize the directed graph
    G = nx.DiGraph()
    
    # 5. Add nodes with objective values
    for schedule in schedule_tuples:
        G.add_node(schedule, objective=schedule_to_obj[schedule])
        if echo:
            print(f"Schedule: {schedule}, Objective: {schedule_to_obj[schedule]:.2f}")
    
    # 6. Define a helper function to find neighbors by moving one patient
    def find_neighbors(schedule: Tuple[int, ...]) -> List[Tuple[int, ...]]:
        neighbors = []
        T = len(schedule)
        for i in range(T):
            if schedule[i] > 0:
                for j in range(T):
                    if i != j:
                        new_schedule = list(schedule)
                        new_schedule[i] -= 1
                        new_schedule[j] += 1
                        neighbors.append(tuple(new_schedule))
        return neighbors
    
    # 7. Add edges based on single patient moves
    for schedule in schedule_tuples:
        neighbors = find_neighbors(schedule)
        for neighbor in neighbors:
            if neighbor in schedule_to_obj:
                # Determine direction based on objective values
                if schedule_to_obj[schedule] > schedule_to_obj[neighbor]:
                    G.add_edge(schedule, neighbor)
                elif schedule_to_obj[neighbor] > schedule_to_obj[schedule]:
                    G.add_edge(neighbor, schedule)
                # If equal, no edge is added
    
    # 8. Assign positions to nodes based on objective values
    # Sort schedules by objective value
    sorted_schedules = sorted(schedule_tuples, key=lambda x: schedule_to_obj[x])
    
    positions = {}
    spacing = 1  # Base spacing between nodes on x-axis
    y_offset = 0
    y_increment = 0.2  # Increment for spreading nodes vertically
    
    for idx, schedule in enumerate(sorted_schedules):
        x = schedule_to_obj[schedule] * spacing
        y = y_offset
        positions[schedule] = (x, y)
        y_offset += y_increment  # Spread vertically to prevent overlap
    
    # Assign positions to graph nodes
    nx.set_node_attributes(G, positions, 'pos')
    
    # 9. Normalize objective values for color mapping
    min_obj = min(objective_values)
    max_obj = max(objective_values)
    if max_obj - min_obj == 0:
        normalized_values = [0.5 for _ in objective_values]
    else:
        normalized_values = [(obj - min_obj) / (max_obj - min_obj) for obj in objective_values]
    
    schedule_to_color = {schedule: norm for schedule, norm in zip(schedule_tuples, normalized_values)}
    
    # 10. Create edge traces with arrows and annotations
    edge_traces = []
    arrow_annotations = []
    
    for edge in G.edges():
        source, target = edge
        source_x, source_y = positions[source]
        target_x, target_y = positions[target]
        
        # Draw the edge as a line
        edge_traces.append(
            go.Scatter(
                x=[source_x, target_x, None],
                y=[source_y, target_y, None],
                mode='lines',
                line=dict(width=1, color='#888'),
                hoverinfo='none'
            )
        )
        
        # Calculate the difference in objective values
        obj_diff = schedule_to_obj[source] - schedule_to_obj[target]
        
        # Add arrow annotation
        arrow_annotations.append(dict(
            ax=source_x,
            ay=source_y,
            x=target_x,
            y=target_y,
            xref='x',
            yref='y',
            axref='x',
            ayref='y',
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1,
            arrowcolor="#000000",
            opacity=0.5
        ))
        
        # Calculate midpoint for the difference label
        mid_x = (source_x + target_x) / 2
        mid_y = (source_y + target_y) / 2
        
        # Add text annotation for the difference
        arrow_annotations.append(dict(
            x=mid_x,
            y=mid_y,
            text=f"{obj_diff:.2f}",
            showarrow=False,
            xref='x',
            yref='y',
            font=dict(color='blue', size=10),
            align='center'
        ))
    
    # 11. Create node traces
    node_x = []
    node_y = []
    node_text = []
    node_color = []
    
    for schedule in schedule_tuples:
        x, y = positions[schedule]
        node_x.append(x)
        node_y.append(y)
        node_text.append(str(schedule))
        node_color.append(schedule_to_color[schedule])
    
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        text=node_text,
        textposition="top center",
        hovertext=[f"Objective: {obj:.2f}" for obj in objective_values],
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale=[
                [0.0, 'rgb(165,0,38)'],    # Deep Red
                [1.0, 'rgb(255,255,255)']  # White
            ],
            reversescale=True,
            color=node_color,
            size=10,
            colorbar=dict(
                thickness=15,
                title=dict(text='Objective Value', side='right'),
                xanchor='left',
            ),
            line_width=2
        )
    )
    
    # 12. Assemble the figure with edge traces and node traces
    fig = go.Figure(data=edge_traces + [node_trace],
                    layout=go.Layout(
                        title=dict(
                            text="Network of Schedules",
                            font=dict(size=16)
                        ),
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        annotations=arrow_annotations + [dict(
                            text="Created with NetworkX and Plotly",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002
                        )],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                    )
                   )
    
    return fig

In [ ]:
# Create_schedule_network function test

def create_schedule_network_from_lists_test():
  
    schedules = [
        [3, 2, 1, 0, 0, 0],
        [2, 3, 1, 0, 0, 0],
        [3, 1, 2, 0, 0, 0],
        [4, 0, 1, 0, 0, 0],
        # Add more schedules as needed
    ]
    
    objective_values = [
        10.5,
        9.8,
        11.2,
        8.7,
        # Corresponding objective values
    ]
    
    # Create the network graph
    fig = create_schedule_network(schedules, objective_values, echo=True)
    
    # Display the graph
    fig.show()
    
create_schedule_network_from_lists()

In [ ]:
def local_search_w_intermediates(
    x: List[int],
    d: int,
    q: float,
    convolutions: any,  # Replace 'any' with the actual type if known
    w: float,
    v_star: any,        # Replace 'any' with the actual type if known
    size: int = 2,
    echo: bool = False
) -> Tuple[np.ndarray, float, List[Tuple[np.ndarray, float]]]:
    """
    Performs a local search to find an optimal schedule, saving intermediate solutions.

    Parameters:
    - x (List[int]): Initial schedule.
    - d (int): Duration threshold.
    - q (float): No-show probability.
    - convolutions (any): Precomputed convolutions (replace 'any' with actual type).
    - w (float): Weight for the objective function.
    - v_star (any): Some precomputed value or structure (replace 'any' with actual type).
    - size (int): Maximum number of patients to switch in the neighborhood.
    - echo (bool): If True, prints debug information.

    Returns:
    - x_star (np.ndarray): The best schedule found.
    - c_star (float): The cost of the best schedule.
    - intermediates (List[Tuple[np.ndarray, float]]): List of intermediate solutions and their costs.
    """
    
    # Initialize the best solution found so far 'x_star' to the input vector 'x'
    x_star = np.array(x).flatten()  # Ensure it's a 1D array

    # Calculate initial objectives and cost
    objectives_star = calculate_objective_serv_time_lookup(x_star, d, q, convolutions)
    c_star = w * objectives_star[0] + (1 - w) * objectives_star[1]

    # Initialize list of intermediate solutions with the initial solution
    intermediates = [(x_star.copy(), c_star)]

    # Set the value of 'T' to the length of the input vector 'x'
    T = len(x_star)

    # Outer loop for the number of patients to switch
    t = 1
    while t < size:
        if echo:
            print(f'Running local search with t = {t}')

        # Generate the neighborhood of the current best solution 'x_star' with 't' patients switched
        ids_gen = powerset(range(T), t)
        neighborhood = get_neighborhood(x_star, v_star, ids_gen)
        
        if echo:
            print(f"Switching {t} patient(s). Size of neighborhood: {len(list(ids_gen))}")

        # Flag to track if a better solution is found
        found_better_solution = False

        for neighbor in neighborhood:
            # Calculate objectives for the neighbor
            objectives = calculate_objective_serv_time_lookup(neighbor, d, q, convolutions)
            cost = w * objectives[0] + (1 - w) * objectives[1]

            # Compare scalar costs
            if cost < c_star:
                x_star = neighbor
                c_star = cost
                intermediates.append((x_star.copy(), c_star))
                
                if echo:
                    print(f"Found better solution: {x_star}, cost: {c_star}")

                # Set the flag to restart the outer loop
                found_better_solution = True
                break  # Break out of the inner loop to restart search

        # If a better solution was found, restart the search from t = 1
        if found_better_solution:
            t = 1  # Restart search with t = 1
        else:
            t += 1  # Move to the next neighborhood size if no better solution was found

    # Return the best solution found 'x_star', its cost, and the list of intermediates
    return x_star, c_star, intermediates

In [ ]:
from functions import local_search, calculate_objective_serv_time_lookup

def test_local_search_w_intermediates():
    """
    Test the local_search_w_intermediates function against the baseline local_search.
    """

    # Test parameters
    T = 10  # Number of time slots
    N = 5   # Number of patients
    x_initial = build_welch_bailey_schedule(N, T)
    d = 2    # Duration threshold
    q = 0.1  # No-show probability
    w = 0.1  # Weight for waiting time in the objective
    v_star = get_v_star(T)
    convolutions = compute_convolutions([0.3, 0.2, 0.1, 0.05, 0.15, 0.2], N, q)  # Example service time probabilities

    # Run the baseline local_search
    x_star_baseline, obj_baseline = local_search(
        x=x_initial,
        d=d,
        q=q,
        convolutions=convolutions,
        w=w,
        v_star=v_star,
        size=2,
        echo=False
    )

    # Run the local_search_w_intermediates
    x_star_intermediates, obj_intermediates, intermediates = local_search_w_intermediates(
        x=x_initial,
        d=d,
        q=q,
        convolutions=convolutions,
        w=w,
        v_star=v_star,
        size=2,
        echo=False
    )

    # Assertions
    assert np.allclose(x_star_baseline, x_star_intermediates), "Final solutions do not match!"
    assert np.isclose(obj_baseline, obj_intermediates), "Final objective values do not match!"
    assert intermediates, "Intermediates list should not be empty!"
    assert isinstance(intermediates, list), "Intermediates should be a list!"
    assert all(len(sol) == 2 for sol in intermediates), "Each intermediate entry must be a tuple with (solution, cost)!"

    # Print results
    print("Baseline solution:", x_star_baseline)
    print("Baseline cost:", obj_baseline)
    print("Intermediates:")
    for i, (sol, cost) in enumerate(intermediates):
        print(f"Step {i}: Solution={sol}, Cost={cost}")

    print("Test passed: local_search_w_intermediates matches local_search and captures intermediates correctly.")

# Run the test
test_local_search_w_intermediates()